# Tratamento estatístico de dados climatológicos
Baseado nos dados do INMET: https://portal.inmet.gov.br/dadoshistoricos

In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import os.path
import progressbar
from ipywidgets import interactive, Dropdown
from urllib.request import urlretrieve
from zipfile import ZipFile 
import matplotlib.pyplot as plt
from scipy import stats

def progresso(block_num, block_size, total_size):
  """Exibe barra de progresso para uso com urlretrieve
  """
  global pbar
  if pbar is None:
    pbar = progressbar.ProgressBar(maxval=total_size)
    pbar.start()  
  downloaded = block_num * block_size
  if downloaded < total_size:
    pbar.update(downloaded)
  else:
    pbar.finish()
    pbar = None
        
def getLocal(local: str, ano: int) -> str:
  """Transcreve o local para o nome do arquivo, de acordo com o período
  """
  s = ''
  match local:
    case 'Bauru':
      s = 'INMET_SE_SP_A705_BAURU'
    case 'Rio de Janeiro':
      if ano > 2020:
        s = 'INMET_SE_RJ_A652_RIO DE JANEIRO - FORTE DE COPACABANA'
      else:
        s = 'INMET_SE_RJ_A652_FORTE DE COPACABANA'
    case 'Teresópolis':
      if ano > 2020:
        s = 'INMET_SE_RJ_A618_TERESOPOLIS-PARQUE NACIONAL'
      else:
        s = 'INMET_SE_RJ_A618_TERESOPOLIS'
    case _:
      raise ValueError('Localização inválida') 
  if ano < 2021:
    return f'{ano}/{s}'
  else:
    return s

def readEstacao(local: str, anoInic: int, anoFim: int):
    url = f'https://portal.inmet.gov.br/uploads/dadoshistoricos/{anoInic}.zip'
    arqzip = f'{anoInic}.zip'
    strlocal = getLocal(local, anoInic)
    arq = f'{strlocal}_01-01-{anoInic}_A_31-12-{anoInic}.CSV'
    if not(os.path.isfile(arqzip)):
        urlretrieve(url, arqzip, progresso)
    with ZipFile(arqzip) as zObject:
        zObject.extract(arq) 
    df = pd.read_csv(arq, sep=';', skiprows=8, encoding='latin1', decimal=',')
    df = df.replace(-9999, np.NaN)
    df = df.rename(columns={
        "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)" : "precTotal", 
        "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)": "pMed", 
        "PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)": "pMax",
        "PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)": "pMin",
        "RADIACAO GLOBAL (KJ/m²)": "rad",
        "RADIACAO GLOBAL (Kj/m²)": "rad",
        "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)": "tMed",
        "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": "tMax", 
        "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": "tMin",
        "TEMPERATURA DO PONTO DE ORVALHO (°C)": "tOrv",
        "TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)": "tOrvMax",
        "TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)": "tOrvMin",
        "UMIDADE RELATIVA DO AR, HORARIA (%)" : "umid",
        "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)" : "umidMax",
        "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)" : "umidMin",
        "VENTO, DIREÇÃO HORARIA (gr) (° (gr))": "dirVento",
        "VENTO, RAJADA MAXIMA (m/s)": "ventoRaj", 
        "VENTO, VELOCIDADE HORARIA (m/s)": "ventoHor"})
    dfTarSeco = df['tMed']
    dist = stats.norm
    res = stats.fit(dist, dfTarSeco[np.isfinite(dfTarSeco)], [(0, 40), (0.5, 5)])
    fig, ax = plt.subplots(1,1)
    ax.hist(dfTarSeco, bins=np.arange(10., 40., 1.), density=True);
    x = np.linspace(10, 40, 100)
    ax.plot(x, stats.norm.pdf(x, *res.params))
    plt.xlabel('Temperatura')
    plt.ylabel('Frequência')
    plt.title('Temperatura do ar - bulbo seco')
    plt.show()

    # TODO: tratar os dados zerados
    dfVento = df['ventoHor']
    dist = stats.weibull_min
    res = stats.fit(dist, dfVento[np.isfinite(dfVento)], [(0, 10), (0.1, 2), (0.1, 10)])
    fig, ax = plt.subplots(1,1)
    ax.hist(dfVento, bins=np.arange(0., 5.0, 0.1), density=True);
    x = np.linspace(0, 5, 100)
    ax.plot(x, stats.weibull_min.pdf(x, *res.params))
    plt.xlabel('Velocidade (m/s)')
    plt.ylabel('Frequência')
    plt.title('Velocidade do vento horária')
    plt.show()
    print(res)

    # Testando Seaborn
    # TODO: extrair parâmetros para fazer uso prático
    fig, ax = plt.subplots(1,1)
    sns.kdeplot(dfVento, color='b', fill=True)
    plt.xlabel('Velocidade (m/s)')
    plt.ylabel('Frequência')
    plt.title('Velocidade do vento horária')
    plt.show()

    dfRaj = df['ventoRaj']
    dist = stats.gumbel_r
    res = stats.fit(dist, dfRaj[np.isfinite(dfRaj)], [(0, 20), (0.5, 5)])
    fig, ax = plt.subplots(1,1)
    ax.hist(dfRaj, bins=np.arange(0., 15.5, 0.5), density=True);
    x = np.linspace(0, 15, 100)
    ax.plot(x, stats.gumbel_r.pdf(x, *res.params))
    plt.xlabel('Velocidade (m/s)')
    plt.ylabel('Frequência')
    plt.title('Velocidade do vento, rajada máxima')
    plt.show()
    print(res)

    df['dra'] = df['pMed']/1013 * (293)/(273 + df['tMed'])
    dfDra = df['dra']
    dist = stats.norm
    res = stats.fit(dist, dfDra[np.isfinite(dfDra)], [(0.5, 1), (0.01, 2)])
    fig, ax = plt.subplots(1,1)
    ax.hist(dfDra, bins=np.arange(0.8, 1, 0.01), density=True);
    x = np.linspace(0.8, 1, 100)
    ax.plot(x, stats.norm.pdf(x, *res.params))
    plt.xlabel('DRA')
    plt.ylabel('Frequência')
    plt.title('Densidade relativa do ar média')
    plt.show()
    print(res)

    print(df)

    # Teste de correlação
    df['date'] = df.iloc[:,0] + ' ' + df.iloc[:,1]
    df['date'] = pd.to_datetime(df['date'])
    print(df.corr())
    df.groupby(df['date'].dt.hour)['ventoRaj'].mean().plot()
    plt.xlabel("Hora (UTC)");
    plt.ylabel("Rajada de vento média (m/s)");
    df.groupby(df['date'].dt.hour)['precTotal'].mean().plot()
    plt.xlabel("Hora (UTC)");
    plt.ylabel("Precipitação média (mm)");
    df.groupby(df['date'].dt.hour)['tempMax'].mean().plot()
    plt.xlabel("Hora (UTC)");
    plt.ylabel("Média da temperatura máxima da hora anterior (°C)");

# Teste de combobox - a ideia é selecionar um ano e uma estação, baixar o zip do ano e extrair
# Exemplo 2001:
# 2001/INMET_SE_SP_A705_BAURU_30-08-2001_A_31-12-2001.CSV
# Exemplo 2008:
# 2008/INMET_SE_SP_A705_BAURU_01-01-2008_A_31-12-2008.CSV
# Exemplo 2023:
# INMET_SE_SP_A705_BAURU_01-01-2023_A_31-12-2023.CSV
pbar = None
dpEstacoes = Dropdown(
    options=['Teresópolis', 'Rio de Janeiro', 'Bauru'],
    description='Estação:',
)
dpAnoInic = Dropdown(
    options=range(2010, 2024),
    description='Ano inicial:',
)
dpAnoFim = Dropdown(
    options=range(2010, 2024),
    description='Ano final:',
)
interactive_load = interactive(readEstacao, local=dpEstacoes, anoInic=dpAnoInic, anoFim=dpAnoFim)
interactive_load

interactive(children=(Dropdown(description='Estação:', options=('Teresópolis', 'Rio de Janeiro', 'Bauru'), val…

## Testes de correlação

TODO: correlacionar os dados para tentar preencher os buracos em algumas séries